In [1]:
import edgar
import urllib
import os
import pandas as pd
import numpy as np
import time
import pickle
import yfinance as yf
import time
import gc

# Scraping EDGAR

In [3]:
# https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
df_sp500 = pd.read_csv("./SP500_Companies",sep='\t')
# df_sp500.CIK = df_sp500.CIK.map(lambda item: str(item).zfill(10))
df_sp500.CIK = df_sp500.CIK.map(lambda item: str(item))
df_sp500

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [4]:
# with open(f"./cik-lookup-data.txt") as f:
#     with open(f"./cik-lookup-data_lower.txt","w") as f2:
#         for line in f:
#             f2.write(line.lower())

In [5]:
# cik2company = {}
# with open(f"./cik-lookup-data_lower.txt") as f:
#     i = 0
#     for line in f:
#         items = line.split(":")
#         try:
#             cik2company[items[1]]
#         except:
#             cik2company[items[1]] = items[0]
            # print([items[1],items[0]])
            # i += 1
            # if i == 100:
            #     break

In [6]:
SP500_Symbols = df_sp500.Symbol.to_list()
SP500_CIK = df_sp500.CIK.to_list()
SP500_Companies = df_sp500.Security.to_list()

In [7]:
symbol2company = dict(zip(SP500_Symbols,SP500_CIK))

In [8]:
len(symbol2company)

505

In [9]:
# for symbol in SP500_Symbols:
#     try:
#         os.mkdir(f"./data/{symbol}")
#     except:
#         pass

In [10]:
# https://github.com/edgarminers/python-edgar
# os.mkdir("./data/_index")
# edgar.download_index(f'./data/_index', 2010,skip_all_present_except_last=False)

In [11]:
years = np.arange(2010,2020+1,1)
quarters = np.arange(1,4+1,1)

In [11]:
# os.mkdir("./data/_index_lowercase")
# for file in os.listdir("./data/_index"):
#     with open(f"./data/_index/{file}") as f:
#         with open(f"./data/_index_lowercase/{file}","w") as f2:
#             for line in f:
#                 f2.write(line.lower())

In [120]:
to_do_list = []
selectedreport = '10-Q'
for selectedsymbol in SP500_Symbols:
    selectedcompany = symbol2company[selectedsymbol]
    for year in years:
        for quarter in quarters:
            if not os.path.isfile(f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm'):
                to_do_list.append([selectedsymbol,selectedcompany,selectedreport,year,quarter])

print(len(to_do_list))
# print(to_do_list)

7739


In [45]:
to_do_list[0]

['MMM', '66740', '10-Q', 2010, 1]

In [46]:
%%time
counter = 0
for to_do in to_do_list:
    selectedsymbol = to_do[0]
    selectedcompany = to_do[1]
    selectedreport = to_do[2]
    year = to_do[3]
    quarter = to_do[4]
    counter+=1
print(counter)
print(selectedsymbol)
print(selectedcompany)
print(selectedreport)
print(year)
print(quarter)
    

8675
ZTS
1555280
10-Q
2020
1
Wall time: 3.17 ms


In [18]:
selectedsymbol

'ZTS'

In [19]:
# %%time
# csvs = {}
# for year in years:
#     for quarter in quarters:
#         csv = pd.read_csv(f'./data/_index_lowercase/{year}-QTR{quarter}.tsv', sep='\t',  lineterminator='\n', names=None) 
#         csvs[(year,quarter)] = csv
# pickle.dump(csvs,open('csvs','wb'))

Wall time: 21.5 s


In [33]:
%%time
csvs = pickle.load(open('csvs','rb'))

Wall time: 4.22 s


In [34]:
len(csvs)

44

In [35]:
print(years)
print(quarters)

[2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020]
[1 2 3 4]


In [37]:
len(to_do_list)

10953

In [43]:
%%time
list_of_fails = []
for to_do in to_do_list[:]:
    selectedsymbol = to_do[0]
    selectedcompany = to_do[1]
    selectedreport = to_do[2]
    year = to_do[3]
    quarter = to_do[4]
    # print(to_do)

    try:
        url = "N/A"

        csv = csvs[(year,quarter)]

        csv.columns.values[0] = 'Item'

        # companyreport = csv[(csv['Item'].str.contains(f"/{selectedcompany}/")) & (csv['Item'].str.contains(selectedreport.lower()))]
        companyreport = csv[csv['Item'].map(lambda item: (f"/{selectedcompany}/" in item) and (selectedreport.lower() in item))]
        if len(companyreport) == 0:
            continue

        Filing = companyreport['Item'].str.split('|')
        # Filing = companyreport.split('|')

        Filing = Filing.to_list()

        for item in Filing[0]:
            
            if 'html' in item:
                report = item
                
        url = 'https://www.sec.gov/Archives/' + report
        url = url.strip()

        df = pd.read_html(url)

        time.sleep(.12)

        document_index = df[0]
        document_index = document_index.dropna()

        document_name = document_index[document_index['Description'].str.contains(selectedreport)]
        document_name = document_name['Document'].str.split(' ')
        document_name = document_name[0][0]

        report_formatted = report.replace('-','').replace('index.html','')
        url = 'https://www.sec.gov/Archives/' + report_formatted.strip() + '/' + document_name # That's the HTM with all the numbers and text

        urllib.request.urlretrieve(url, f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm')
        time.sleep(.12)
    except:
        # time.sleep(1)
        failure = [selectedsymbol,selectedcompany,selectedreport,year,quarter,url]
        list_of_fails.append(failure)
        print(failure)
# pickle.dump(list_of_fails,open('list_of_fails','wb'))

data/100517/0001193125-16-651221-index.html']
['UAL', '100517', '10-Q', 2019, 2, 'https://www.sec.gov/Archives/edgar/data/100517/000010051719000014/ual_03311910q.htm']
['UAL', '100517', '10-Q', 2020, 2, 'https://www.sec.gov/Archives/edgar/data/100517/000010051720000037/ual10q03312020.htm']
['UPS', '1090727', '10-Q', 2010, 2, 'https://www.sec.gov/Archives/edgar/data/1090727/0001193125-10-112914-index.html']
['UPS', '1090727', '10-Q', 2012, 3, 'https://www.sec.gov/Archives/edgar/data/1090727/0001090727-12-000015-index.html']
['UPS', '1090727', '10-Q', 2015, 3, 'https://www.sec.gov/Archives/edgar/data/1090727/0001090727-15-000035-index.html']
['UPS', '1090727', '10-Q', 2018, 3, 'https://www.sec.gov/Archives/edgar/data/1090727/0001090727-18-000032-index.html']
['URI', '1067701', '10-Q', 2010, 3, 'https://www.sec.gov/Archives/edgar/data/1067701/0001193125-10-161856-index.html']
['URI', '1067701', '10-Q', 2012, 3, 'https://www.sec.gov/Archives/edgar/data/1067701/0001067701-12-000017-index.ht

In [ ]:
# os.system(r"%windir%\System32\rundll32.exe powrprof.dll,SetSuspendState 0,1,0")

In [53]:
failed = set()
for item in to_do_list:
    failed.add(tuple([item[0],item[1]]))
for item in failed:
    print(item)

('MS', '895421')
('SNA', '91440')
('DXCM', '1093557')
('PHM', '822416')
('RMD', '943819')
('SPGI', '64040')
('MOS', '1285785')
('DAL', '27904')
('MCO', '1059556')
('ADP', '8670')
('TJX', '109198')
('PRU', '1137774')
('DISH', '1001082')
('IT', '749251')
('HUM', '49071')
('FBHS', '1519751')
('MPWR', '1280452')
('KLAC', '319201')
('FLIR', '354908')
('BK', '1390777')
('CDW', '1402057')
('CPRT', '900075')
('MHK', '851968')
('PFE', '78003')
('IPG', '51644')
('CAT', '18230')
('HAL', '45012')
('MA', '1141391')
('ILMN', '1110803')
('MET', '1099219')
('NUE', '73309')
('PENN', '921738')
('FCX', '831259')
('XOM', '34088')
('STZ', '16918')
('HFC', '48039')
('GILD', '882095')
('UNH', '731766')
('WLTW', '1140536')
('AES', '874761')
('LW', '1679273')
('ALXN', '899866')
('AIG', '5272')
('RE', '1095073')
('ISRG', '1035267')
('GIS', '40704')
('LH', '920148')
('AMT', '1053507')
('CINF', '20286')
('HRL', '48465')
('PG', '80424')
('APA', '6769')
('GOOGL', '1652044')
('TPR', '1116132')
('MCD', '63908')
('FMC

In [50]:
# list_of_fails = pickle.load(open('list_of_fails','rb'))
# len(list_of_fails)

2158

In [118]:
%%time

list_of_fails2 = []
count = 0
for fail_item in list_of_fails[:]:
    if fail_item[5][-4:] == 'html':
        count += 1

        selectedsymbol = fail_item[0]
        selectedcompany = fail_item[1]
        selectedreport = fail_item[2]
        year = fail_item[3]
        quarter = fail_item[4]

        try:
            url = "N/A"

            csv = csvs[(year,quarter)]

            csv.columns.values[0] = 'Item'

            # companyreport = csv[(csv['Item'].str.contains(f"/{selectedcompany}/")) & (csv['Item'].str.contains(selectedreport.lower()))]
            companyreport = csv[csv['Item'].map(lambda item: (f"/{selectedcompany}/" in item) and (selectedreport.lower() in item))]
            if len(companyreport) == 0:
                continue

            Filing = companyreport['Item'].str.split('|')
            # Filing = companyreport.split('|')

            Filing = Filing.to_list()

            for item in Filing[0]:
                
                if 'html' in item:
                    report = item
                    
            url = 'https://www.sec.gov/Archives/' + report
            url = url.strip()

            df = pd.read_html(url)

            time.sleep(.12)

            document_index = df[0]
            document_index = document_index.dropna()

            # Method 1
            document_name = document_index[document_index['Description'].str.contains(selectedreport)]

            # Method 2
            if len(document_name) == 0:
                document_name = document_index[document_index['Description'].str.contains(selectedreport.replace("-",""))]

            # Method 3
            if len(document_name) == 0:
                document_name = document_index[document_index['Document'].str.contains(selectedreport.replace("-","").lower() + ".htm")]

            document_name = document_name['Document'].str.split(' ')
            document_name = document_name[0][0]

            report_formatted = report.replace('-','').replace('index.html','')
            url = 'https://www.sec.gov/Archives/' + report_formatted.strip() + '/' + document_name # That's the HTM with all the numbers and text

            urllib.request.urlretrieve(url, f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm')
            time.sleep(.12)
        except:
            failure = [selectedsymbol,selectedcompany,selectedreport,year,quarter,url]
            list_of_fails2.append(failure)
            print(failure)


print(count)
print(len(list_of_fails))
print(len(list_of_fails2))


', '882835', '10-Q', 2011, 4, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-11-000032-index.html']
['ROP', '882835', '10-Q', 2012, 2, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-12-000022-index.html']
['ROP', '882835', '10-Q', 2012, 3, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-12-000034-index.html']
['ROP', '882835', '10-Q', 2012, 4, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-12-000048-index.html']
['ROP', '882835', '10-Q', 2013, 2, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-13-000026-index.html']
['ROP', '882835', '10-Q', 2013, 3, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-13-000042-index.html']
['ROP', '882835', '10-Q', 2013, 4, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-13-000050-index.html']
['ROP', '882835', '10-Q', 2014, 2, 'https://www.sec.gov/Archives/edgar/data/882835/0000882835-14-000020-index.html']
['ROP', '882835', '10-Q', 2014, 3, 'https://www.sec.gov/Archives/edga

In [119]:
# pickle.dump(list_of_fails2,open('list_of_fails2','wb'))

In [121]:
done_list = []
selectedreport = '10-Q'
for selectedsymbol in SP500_Symbols:
    selectedcompany = symbol2company[selectedsymbol]
    for year in years:
        for quarter in quarters:
            if os.path.isfile(f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm'):
                done_list.append([selectedsymbol,selectedcompany,selectedreport,year,quarter])

print(len(done_list))
# print(to_do_list)

14481


# Scraping Stocks

In [124]:
print(len(SP500_Symbols))
print(len(years))
print(len(quarters))

505
11
4


In [131]:
tickers = " ".join(SP500_Symbols)
tickers

'MMM ABT ABBV ABMD ACN ATVI ADBE AMD AAP AES AFL A APD AKAM ALK ALB ARE ALXN ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AAL AEP AXP AIG AMT AWK AMP ABC AME AMGN APH ADI ANSS ANTM AON AOS APA AAPL AMAT APTV ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY BKR BLL BAC BK BAX BDX BRK.B BBY BIO BIIB BLK BA BKNG BWA BXP BSX BMY AVGO BR BF.B CHRW COG CDNS CZR CPB COF CAH KMX CCL CARR CTLT CAT CBOE CBRE CDW CE CNC CNP CERN CF SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CTXS CLX CME CMS KO CTSH CL CMCSA CMA CAG COP ED STZ COO CPRT GLW CTVA COST CCI CSX CMI CVS DHI DHR DRI DVA DE DAL XRAY DVN DXCM FANG DLR DFS DISCA DISCK DISH DG DLTR D DPZ DOV DOW DTE DUK DRE DD DXC EMN ETN EBAY ECL EIX EW EA EMR ENPH ETR EOG EFX EQIX EQR ESS EL ETSY EVRG ES RE EXC EXPE EXPD EXR XOM FFIV FB FAST FRT FDX FIS FITB FE FRC FISV FLT FLIR FMC F FTNT FTV FBHS FOXA FOX BEN FCX GPS GRMN IT GNRC GD GE GIS GM GPC GILD GL GPN GS GWW HAL HBI HIG HAS HCA PEAK HSIC HSY HES HPE HLT HFC HOLX HD HON HRL HST HWM HPQ HUM HBAN 

In [137]:
# data = yf.download(tickers, start="2010-01-01", end="2021-3-31")

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [138]:
# pickle.dump(data, open("stock_history","wb"))

In [139]:
data

Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2010-01-04   20.514248   4.496876   38.980095    6.547977         NaN   
2010-01-05   20.291416   5.005957   38.748432    6.559296         NaN   
2010-01-06   20.219320   4.798555   39.086288    6.454964         NaN   
2010-01-07   20.193102   4.939965   39.076641    6.443031         NaN   
2010-01-08   20.186556   4.845690   39.231079    6.485865         NaN   
...                ...        ...         ...         ...         ...   
2021-03-25  121.714798  22.770000  185.649994  120.589996  102.632133   
2021-03-26  125.449112  22.930000  187.320007  121.209999  104.706917   
2021-03-29  125.229446  22.910000  185.059998  121.389999  105.447906   
2021-03-30  124.650322  24.120001  186.070007  119.900002  105.507179   
2021-03-31         NaN        NaN         NaN         NaN         NaN   

                                                                        ...  \
                   ABC        ABMD         ABT         ACN        ADBE  ...   
Date                                                                    ...   
2010-01-04   22.419764    8.740000   20.001102   33.614555   37.090000  ...   
2010-01-05   22.259808    8.530000   19.839502   33.822308   37.700001  ...   
2010-01-06   22.049330    8.400000   19.949684   34.181858   37.619999  ...   
2010-01-07   21.695730    8.400000   20.114946   34.149891   36.889999  ...   
2010-01-08   21.931467    8.230000   20.217783   34.014061   36.689999  ...   
...                ...         ...         ...         ...         ...  ...   
2021-03-25  117.339996  294.140015  118.614487  267.781250  450.989990  ...   
2021-03-26  118.730003  301.399994  121.623436  279.903717  469.089996  ...   
2021-03-29  119.050003  305.769989  121.782860  278.677551  469.320007  ...   
2021-03-30  119.059998  309.880005  119.311928  277.690582  465.459991  ...   
2021-03-31         NaN         NaN         NaN         NaN         NaN  ...   

               Volume                                                          \
                  XEL       XLNX         XOM       XRAY        XYL        YUM   
Date                                                                            
2010-01-04  2670400.0  2824700.0  27809100.0  1051400.0        NaN  2962200.0   
2010-01-05  4321400.0  3469700.0  30174700.0   763400.0        NaN  3298700.0   
2010-01-06  2164500.0  6329200.0  35044700.0  1595100.0        NaN  4178900.0   
2010-01-07  3041700.0  9252700.0  27192100.0  1096100.0        NaN  2452400.0   
2010-01-08  1599100.0  5005800.0  24891800.0   803600.0        NaN  3772300.0   
...               ...        ...         ...        ...        ...        ...   
2021-03-25  3475400.0  1479400.0  31495900.0  1893500.0  1070700.0  1430400.0   
2021-03-26  2643000.0  2360300.0  34226500.0  1131300.0   790900.0  1391400.0   
2021-03-29  2379000.0  1792200.0  21144400.0  1237000.0   808900.0  1717900.0   
2021-03-30  2284400.0  1594700.0  20303400.0  1664600.0   773800.0  1694000.0   
2021-03-31        NaN        NaN         NaN        NaN        NaN        NaN   

                                                        
                  ZBH      ZBRA        ZION        ZTS  
Date                                                    
2010-01-04   782400.0  168800.0   3974600.0        NaN  
2010-01-05  1718100.0  168800.0   5605500.0        NaN  
2010-01-06  1277300.0  385300.0  12615200.0        NaN  
2010-01-07  1683500.0  183600.0  24716800.0        NaN  
2010-01-08  2149500.0  266500.0   6903000.0        NaN  
...               ...       ...         ...        ...  
2021-03-25  1081900.0  349000.0   1194300.0  1656600.0  
2021-03-26   651900.0  261300.0   1257900.0  1877200.0  
2021-03-29   667100.0  235500.0   1059700.0  2044800.0  
2021-03-30   762900.0  251400.0   1225500.0  1298000.0  
2021-03

In [147]:
print(data.columns.levels[0])
print(data.columns.levels[1])

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       ...
       'XEL', 'XLNX', 'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', length=505)


# Extracting Data Out of Scrapes

In [13]:
done_list = []
selectedreport = '10-Q'
for selectedsymbol in SP500_Symbols:
    selectedcompany = symbol2company[selectedsymbol]
    for year in years:
        for quarter in quarters:
            if os.path.isfile(f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm'):
                done_list.append([selectedsymbol,selectedcompany,selectedreport,year,quarter])

print(len(done_list)/len(SP500_Symbols)/len(quarters))

7.168811881188119


In [11]:
15000*10/60/60

41.666666666666664

In [14]:
%%time
dfs = {}
i = 3657
failures = []
for line in done_list[:]:
    t1 = time.time()
    # ['MMM', '66740', '10-Q', 2010, 2]
    selectedsymbol = line[0]
    selectedreport = line[2]
    year = line[3]
    quarter = line[4]
    url = f'./data/{selectedsymbol}/{selectedsymbol}_{year}_{quarter}-{selectedreport}.htm'

    try:
        df = pd.read_html(url)

        dfs[(selectedsymbol,selectedreport,year,quarter)] = df

    except:
        failures.append((i,selectedsymbol,selectedreport,year,quarter))

    print(f"{i} -> {time.time()-t1} s")
    i += 1

809 -> 1.4329874515533447 s
13810 -> 0.7457399368286133 s
13811 -> 1.4692139625549316 s
13812 -> 0.7930383682250977 s
13813 -> 1.6681077480316162 s
13814 -> 0.7149360179901123 s
13815 -> 1.2378289699554443 s
13816 -> 1.2352662086486816 s
13817 -> 9.730022192001343 s
13818 -> 10.813421964645386 s
13819 -> 1.8906991481781006 s
13820 -> 2.709219217300415 s
13821 -> 2.679813861846924 s
13822 -> 3.4421908855438232 s
13823 -> 3.400707483291626 s
13824 -> 2.1375460624694824 s
13825 -> 0.025633573532104492 s
13826 -> 4.348499536514282 s
13827 -> 2.2939183712005615 s
13828 -> 4.797140836715698 s
13829 -> 5.126198768615723 s
13830 -> 2.3660526275634766 s
13831 -> 4.043329238891602 s
13832 -> 4.501244783401489 s
13833 -> 1.876558780670166 s
13834 -> 4.58755350112915 s
13835 -> 5.037110090255737 s
13836 -> 2.4399526119232178 s
13837 -> 4.930513620376587 s
13838 -> 4.884355545043945 s
13839 -> 2.3153653144836426 s
13840 -> 4.010316848754883 s
13841 -> 4.080643892288208 s
13842 -> 1.6371164321899414

In [15]:
# pickle.dump(dfs, open("quarterly_report_history","wb"))

In [18]:
# dfs = pickle.load(open('quarterly_report_history','rb'))

In [25]:
len(dfs)

14477

In [4]:
# %%time
# dfs = pickle.load(open('quarterly_report_history3657','rb'))
# dfs.update(pickle.load(open('quarterly_report_history3657_on','rb')))


Wall time: 13min 9s


In [5]:
# %%time
# pickle.dump(dfs, open("quarterly_report_history","wb"))

Wall time: 10min 2s


# Re Pickle

In [2]:
%%time
dfs = pickle.load(open('quarterly_report_history','rb'))

Wall time: 5min 14s


In [3]:
len(list(dfs.items())[0][1])

68

In [4]:
pickle.HIGHEST_PROTOCOL

4

In [5]:
pickle.DEFAULT_PROTOCOL

3

In [8]:
pickle.dump(dfs, open('quarterly_report_history_p4','wb'),protocol=4)

In [9]:
%%time
dfs = pickle.load(open('quarterly_report_history_p4','rb'))

Wall time: 21min 33s


In [10]:
%%time
pickle.dump(dfs, open('quarterly_report_history_p1','wb'),protocol=1)

Wall time: 21h 2min 53s


In [2]:
%%time
dfs = pickle.load(open('quarterly_report_history_p1','rb'))

Wall time: 6min 3s


# Data Size Reduciton

In [5]:
len(dfs)

14477

In [37]:
len(list(dfs.items())[0][1])

68

In [48]:
list(dfs.items())[0][0]

('MMM', '10-Q', 2010, 2)

In [87]:
%%time
inputs = []
for item in dfs:
    inputs.append(":".join(str(x) for x in item))
print(inputs[0])
print(len(inputs))


MMM:10-Q:2010:2
14477
Wall time: 49.4 ms


In [121]:
%%time
outputs = []
# i = 0
for item in dfs:
    string_list = []
    contents = dfs[item]
    # j = 0
    for content in contents:
        string_list.append(content.to_string())
        # j += 1
        # if j == 20:
        #     break
    final_string = " ".join(string_list)
    outputs.append(final_string)
    # i += 1
    # if i == 15000:
    #     break
# print(outputs[0][1:1000])
print(len(outputs))


14477
Wall time: 1h 31min 10s


In [118]:
# pickle.dump(inputs,open("inputs","wb"))

In [122]:
# %%time
# pickle.dump(outputs,open("outputs","wb"))

Wall time: 1min 39s


In [123]:
%%time
outputs = pickle.load(open('outputs','rb'))

Wall time: 1min 36s


# Stock History Data

In [124]:
stock_history = pickle.load(open("stock_history",'rb'))

In [133]:
stock_history.columns.levels[0]

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [148]:
stock_history['Adj Close']

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,20.514248,4.496876,38.980095,6.547977,NaN,22.419764,8.740000,20.001102,33.614555,37.090000,...,14.165524,19.830650,45.575386,32.887291,NaN,20.231468,55.417038,28.670000,11.648637,NaN
2010-01-05,20.291416,5.005957,38.748432,6.559296,NaN,22.259808,8.530000,19.839502,33.822308,37.700001,...,13.997527,19.580618,45.753330,32.496326,NaN,20.162270,57.171318,28.620001,12.059352,NaN
2010-01-06,20.219320,4.798555,39.086288,6.454964,NaN,22.049330,8.400000,19.949684,34.181858,37.619999,...,14.024410,19.447794,46.148773,32.710423,NaN,20.018137,57.152866,28.400000,13.107989,NaN
2010-01-07,20.193102,4.939965,39.076641,6.443031,NaN,21.695730,8.400000,20.114946,34.149891,36.889999,...,13.963934,19.252451,46.003792,33.138615,NaN,20.012373,58.463970,27.690001,14.576083,NaN
2010-01-08,20.186556,4.845690,39.231079,6.485865,NaN,21.931467,8.230000,20.217783,34.014061,36.689999,...,13.970654,19.533739,45.819229,33.138615,NaN,20.018137,57.235977,27.600000,14.340141,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-25,121.714798,22.770000,185.649994,120.589996,102.632133,117.339996,294.140015,118.614487,267.781250,450.989990,...,66.000000,120.029999,56.180000,60.549999,101.930000,107.379997,157.639999,461.260010,54.740002,152.652649
2021-03-26,125.449112,22.930000,187.320007,121.209999,104.706917,118.730003,301.399994,121.623436,279.903717,469.089996,...,66.309998,123.139999,57.709999,61.279999,104.760002,108.059998,161.320007,476.959991,55.849998,155.917770
2021-03-29,125.229446,22.910000,185.059998,121.389999,105.447906,119.050003,305.769989,121.782860,278.677551,469.320007,...,67.000000,122.230003,57.400002,62.060001,104.269997,109.209999,160.210007,467.070007,53.889999,158.154449


In [178]:
%%time

df_stocks = stock_history['Adj Close'].copy()
df_stocks = df_stocks[0:0]
df_stocks

for index, row in stock_history['Adj Close'].iterrows():
    if index.day == 15 and index.month in [3,6,9,12]:
        df_stocks = df_stocks.append(row)

df_stocks = df_stocks.dropna(axis=1)
df_stocks

Wall time: 237 ms


,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XLNX,XOM,XRAY,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-03-15,21.831623,7.447657,40.621174,6.848741,23.431705,10.350000,20.272953,33.958130,35.490002,21.875849,...,11.220833,46.234886,14.212567,20.728977,43.982063,32.366016,21.852259,53.773556,30.080000,18.571239
2010-06-15,21.104124,9.861077,50.401211,7.945628,27.478739,9.740000,17.970345,31.064312,32.509998,23.142696,...,10.553507,55.883804,14.315210,20.875732,41.749474,29.299294,25.286951,50.191101,27.500000,21.201899
2010-09-15,19.275530,8.399828,55.561707,8.267811,25.678307,10.040000,19.451303,31.950949,32.759998,22.196320,...,11.198768,59.583954,15.799513,20.757133,41.032509,28.552584,26.903006,46.691769,31.700001,18.323788
2010-12-15,25.220085,9.559399,65.665291,9.801922,27.482336,9.220000,18.268618,37.527168,27.719999,28.702520,...,12.537512,71.756393,16.342747,22.616062,48.635223,31.534311,29.058336,49.581734,38.630001,19.391413
2011-03-15,28.719950,8.409253,63.435417,10.568980,31.118685,13.740000,18.258106,40.718197,33.040001,29.222143,...,17.526907,86.308533,16.610018,25.251810,55.383179,33.368744,30.159559,56.017197,35.509998,19.916029
2011-06-15,31.046650,7.758763,55.748959,9.997436,35.143578,15.960000,19.847197,45.472313,31.129999,28.903385,...,14.293878,93.181389,17.130363,27.190594,53.829094,33.865601,32.143921,57.974613,40.740002,19.582691
2011-09-15,24.276295,5.524466,60.887012,12.023236,33.135494,11.850000,20.019785,43.510948,26.120001,27.076778,...,12.623442,107.258888,17.966824,25.792480,50.981449,32.173832,32.062984,52.370125,32.040001,15.317099
2011-12-15,21.923384,5.383055,65.847656,11.594273,30.853142,19.770000,21.621883,46.615009,26.459999,27.035887,...,12.080142,78.470772,19.170029,25.410080,55.456764,32.724586,34.553699,44.078808,36.720001,13.080435
2012-03-15,30.056978,7.145980,86.870781,17.916140,32.785091,21.959999,23.485102,52.843628,34.169998,31.844351,...,16.228251,96.480072,19.331430,30.366581,59.982555,37.448090,41.432911,58.971786,41.279999,19.424265


In [179]:
# pickle.dump(df_stocks,open('df_stocks','wb'))

# Stock Increase Data

In [194]:
%%time

df_inc = df_stocks.copy()
df_inc = df_inc[0:0]
df_inc

for i in np.arange(len(df_stocks)-1):
    new_row = (df_stocks.iloc[i+1]-df_stocks.iloc[i])/df_stocks.iloc[i]
    new_row.name = df_stocks.iloc[i+1].name
    df_inc = df_inc.append(new_row)

df_inc

Wall time: 144 ms


,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,WY,WYNN,XEL,XLNX,XOM,XRAY,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2010-06-15,-0.033323,0.324051,0.240762,0.160159,0.172716,-0.058937,-0.113580,-0.085217,-0.083967,0.057911,...,-0.059472,0.208693,0.007222,0.007080,-0.050761,-0.094751,0.157178,-0.066621,-0.085771,0.141652
2010-09-15,-0.086646,-0.148184,0.102388,0.040548,-0.065521,0.030801,0.082411,0.028542,0.007690,-0.040893,...,0.061142,0.066211,0.103687,-0.005681,-0.017173,-0.025486,0.063909,-0.069720,0.152727,-0.135748
2010-12-15,0.308399,0.138047,0.181844,0.185552,0.070255,-0.081673,-0.060802,0.174524,-0.153846,0.293121,...,0.119544,0.204291,0.034383,0.089556,0.185285,0.104429,0.080115,0.061895,0.218612,0.058264
2011-03-15,0.138773,-0.120316,-0.033958,0.078256,0.132316,0.490239,-0.000575,0.085032,0.191919,0.018104,...,0.397957,0.202799,0.016354,0.116543,0.138746,0.058173,0.037897,0.129795,-0.080766,0.027054
2011-06-15,0.081013,-0.077354,-0.121170,-0.054078,0.129340,0.161572,0.087035,0.116757,-0.057809,-0.010908,...,-0.184461,0.079631,0.031327,0.076778,-0.028061,0.014890,0.065795,0.034943,0.147283,-0.016737
2011-09-15,-0.218070,-0.287971,0.092164,0.202632,-0.057139,-0.257519,0.008696,-0.043133,-0.160938,-0.063197,...,-0.116864,0.151076,0.048829,-0.051419,-0.052902,-0.049955,-0.002518,-0.096671,-0.213549,-0.217825
2011-12-15,-0.096922,-0.025597,0.081473,-0.035678,-0.068879,0.668354,0.080026,0.071340,0.013017,-0.001510,...,-0.043039,-0.268398,0.066968,-0.014826,0.087783,0.017118,0.077682,-0.158322,0.146067,-0.146024
2012-03-15,0.371001,0.327495,0.319269,0.545258,0.062618,0.110774,0.086173,0.133618,0.291383,0.177855,...,0.343382,0.229503,0.008419,0.195060,0.081609,0.144341,0.199088,0.337872,0.124183,0.484986
2012-06-15,-0.125595,0.587071,-0.218071,-0.019520,-0.012858,0.008652,0.066394,-0.061475,-0.052092,-0.064777,...,-0.055113,-0.218703,0.104928,-0.111527,-0.026678,-0.048506,-0.048942,-0.007710,-0.217781,-0.154538


In [195]:
# pickle.dump(df_inc,open('df_inc','wb'))

In [202]:
df_inc_boolean = df_inc.copy()
df_inc_boolean = df_inc_boolean.applymap(lambda x: x>=.02)

In [203]:
# pickle.dump(df_inc_boolean,open('df_inc_boolean','wb'))